# Agentic RAG - 관련성 평가 도구를 추가

이번 챕터에서는 문서 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들어 보겠습니다.

**참고**

- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 **Agentic RAG** 라고 부릅니다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.


In [18]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [19]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agentic-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agentic-RAG


### 문서 기반 문서 검색 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.

**실습에 활용한 문서**

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


이 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축합니다.

여기서는 PDF 문서를 `FAISS` DB 에 저장하고 조회하는 retriever 를 생성합니다.


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

이 함수는 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용됩니다.


In [21]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12}, page_content='SPRi AI Brief |  \n2023-12 월호\n10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개\nn삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스 ’를 공개\nn삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원\nn삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스 ’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환합니다.

In [22]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

## Relevance Checker

`langchain_teddynote` 패키지에서 제공하는 `OpenAIRelevanceGrader` 를 사용하여 관련성 체크를 수행합니다.

- `Evaluator` 는 **CH15-Evaluation** 챕터를 참고해 주세요.

In [23]:
from langchain_teddynote.evaluator import OpenAIRelevanceGrader
from langchain_openai import ChatOpenAI

retrieval_question_evaluator = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-question"
).create()

`context` & `input` 을 입력으로 전달하여 관련성 체크를 수행합니다.

In [24]:
retrieval_question_evaluator.invoke(
    {
        "context": [
            "삼성전자는 지난 2023년 자체적으로 만든 생성형 AI 가우스 를 발표했다."  # 컨텍스트
        ],
        "input": "삼성전자의 생성형 AI 이름은 빅스비 입니까?",  # 입력
    }
)

GradeRetrievalQuestion(score='no')

In [25]:
retrieval_question_evaluator.invoke(
    {
        "context": [
            "삼성전자는 지난 2023년 자체적으로 만든 생성형 AI 가우스 를 발표했다."  # 컨텍스트
        ],
        "input": "삼성전자의 생성형 AI 이름은 가우스 입니다.",  # 입력
    }
)

GradeRetrievalQuestion(score='yes')

### Relevance Check(관련성 체크) 를 도구로 변환


In [26]:
from langchain_core.tools import tool


@tool
def check_relevance(input: str, context: list[str]) -> bool:
    """This is a tool for checking relevance between the user's query and retrieved context."""
    return retrieval_question_evaluator.invoke({"context": context, "input": input})

아래는 테스트용 가짜 관련성 체크 

In [48]:
from langchain_core.tools import tool


@tool
def check_relevance_fake(input: str, context: list[str]) -> bool:
    """This is a tool for checking relevance between the user's query and retrieved context."""

    if len(input) % 2 == 0:
        return "no"
    else:
        return "no"

In [49]:
# tools 리스트에 retriever_tool을 추가합니다.
# tools = [retriever_tool, check_relevance]
tools = [retriever_tool, check_relevance_fake]

## Agent 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. 

먼저, Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

**참고**
- 멀티턴 대화를 지원하지 않는다면 "chat_history" 를 제거해도 좋습니다.

In [50]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, load_prompt

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            load_prompt("prompts/agentic-rag-prompt-relevance-v2.yaml").template,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [51]:
# 프롬프트 출력
prompt.pretty_print()

================================ System Message ================================

You are a question answering assistant. Your task is to answer the user's question based on the provided context.

You should follow the below guidelines step by step:
1. make sure to use the `pdf_search` tool for searching information from the PDF document.
2. Then, use the `relevance_check` tool for checking the relevance between the user's query and retrieved context.
3. If the result of `relevance_check` is **no**, you should generate another query and use the `pdf_search` tool again.
4. You should include `relevance_check` tool call in your response.

Remember:
- If the result of `relevance_check` is **no**, you should re-generate another query and use the `pdf_search` tool again.
- You can do this process up to 20 times.


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================


다음으로는 Tool Calling Agent 를 생성합니다.

In [52]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성합니다.

**참고**

- `verbose=False` 로 설정하여 중간 단계 출력을 생략하였습니다.

In [53]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, max_iterations=20
)

## 에이전트 실행하기

이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의입니다(이전 상호작용을 기억하지 않습니다).


In [54]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공합니다.


In [55]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`




Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`


SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 

Stopping agent prematurely due to triggering stop condition


[도구 호출]
Tool: check_relevance_fake
input: 삼성전자가 자체 개발한 생성형 AI 관련된 정보
context: ['삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 AI 모델 ‘삼성 가우스’를 공개', '삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유', '삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 이미지 모델의 3개 모델로 구성', '삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유']
Log: 
Invoking: `check_relevance_fake` with `{'input': '삼성전자가 자체 개발한 생성형 AI 관련된 정보', 'context': ['삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 AI 모델 ‘삼성 가우스’를 공개', '삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유', '삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 이미지 모델의 3개 모델로 구성', '삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유']}`




Invoking: `check_relevance_fake` with `{'input': '삼성전자가 자체 개발한 생성형 AI 관련된 정보', 'context': ['삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성

**[참고]**

- 관련성 체크의 결과가 `no` 인 경우, 질문을 재구성하여 다시 검색을 수행합니다.
- 하지만 이 과정에서 반복 loop 를 다시 수행하지 않고 중간에 종료 되는 경우가 있습니다. 왜 그럴까요?